______

# Land Cover Classification for use in the CAPRA Model 

## Nonlinear filters

_____

In previous noteboooks, we applied convolutional filters.  Those filter compute a linear combination of pixels in a neighborhood(specified by the window shape) according to the weights specified by the kernel. Non-linear functions applied to a neighborhood are also useful. [Median](https://en.wikipedia.org/wiki/Median_filter) and [mode](https://www.cs.washington.edu/research/metip/tutor/tutor.Filtering.html)  are examples of non linear filters.  <br>

Median filters are useful for removing noise in an image, especially pixel values with no relation to the image scene. Suppose that random pixels in the image are saturated by anomalously high or low values that result from some noise process. Filtering the image with a mean filter (previous notebook) would result in pixels values getting polluted by noisy data. The median filter ranks the pixels in the neighborhood from lowest to highest and selects the median value, which is then placed in the central value of the mask. 

An approach to apply nonlinear filters in GEE is by using the function *reduceNeighborhood()*. 


In [1]:
import ee
from IPython.display import Image

ee.Initialize()

# load the image collection and filter
l8sr = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')

CBD = ee.FeatureCollection("USDOS/LSIB/2013")
boundary = CBD.filterMetadata('name', 'equals', 'DOMINICAN REPUBLIC')

visimage = ee.Image(l8sr
            .filterDate('2019-01-01', '2019-04-30')
            .filterBounds(boundary)
            .sort('CLOUD_COVER')   
            .select('B[2-5]')   
            .first()
)

uniformKernel = ee.Kernel.square(1)  # this means a 3x3 window

median = visimage.reduceNeighborhood(
  reducer = ee.Reducer.median(), 
  kernel = uniformKernel
)
 
# band names of the result image will have the suffix "_median"     
Image(url=median.getThumbUrl({'min': 0, 'max': 2048,'bands': 'B4_median,B3_median,B2_median'}))
 

In [2]:
uniformKernel = ee.Kernel.square(2)  # this means a 5x5 window

median = visimage.reduceNeighborhood(
  reducer = ee.Reducer.median(), 
  kernel = uniformKernel
)
 
# band names of the result image will have the suffix "_median"     
Image(url=median.getThumbUrl({'min': 0, 'max': 2048,'bands': 'B4_median,B3_median,B2_median'}))
 

Filters can help enhancing visually the image and reducing noise. However, when the interest rely on objects (connected group of pixels)  present in the image rather than individual pixels, we may use [morphological operations](https://en.wikipedia.org/wiki/Mathematical_morphology).  These operations are useful to clean objects that can have gaps or are not well defined because of noise. 

Dilation and erosion are commonly applied to remote sensing images. The following lines apply dilation and erosion to a multi-spectral image. Later those operations are applied to a classified (categorical) image as well. In these examples, the *structuring element* is a square of size 3x3 pixels (square kernel of radio 1 pixel). *Opening* and *closing* can be executed by combining erosion and dilation. <br>


In [3]:
# dilation 
# Dilate by taking the max in each 3x3 neighborhood.
imagemax = visimage.reduceNeighborhood(
  reducer = ee.Reducer.max(), 
  kernel = ee.Kernel.square(1)
)
Image(url=imagemax.getThumbUrl({'min': 0, 'max': 2048,'bands': 'B4_max,B3_max,B2_max'}))

In [4]:
# erosion
# Dilate by taking the max in each 3x3 neighborhood.
imagemin = visimage.reduceNeighborhood(
  reducer = ee.Reducer.min(), 
  kernel = ee.Kernel.square(1)
)
Image(url=imagemin.getThumbUrl({'min': 0, 'max': 2048,'bands': 'B4_min,B3_min,B2_min'}))

You may want to try with a different structuring element, for example a disk by using a  kernel circle.